In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


In [ ]:
from retrieval import get_data
df = get_data(fname='stats.html')

In [ ]:
df.loc[df['reads'] > 1000]


In [ ]:
@interact
def show_articles_more_than(column='claps', x=5000):
    return df.loc[df[column] > x]


In [ ]:
@interact
def show(column=['claps', 'views', 'fans', 'reads'], x=(10, 100000, 10)):
    return df.loc[df[column] > x]

In [ ]:
# @interact
# def show(column=['claps', 'views', 'fans', 'reads'], x=(10, 100000, 10), ascending=[True, False]):
#     return df.loc[df[column] > x].sort_values(by=column, ascending=ascending)

In [ ]:
import os
from IPython.display import Image, display, HTML

In [ ]:
@interact
def show_images(file=os.listdir('images/')):
    display(Image('images/' + file))

In [ ]:
@interact
def correlations(column1=list(df.select_dtypes('number').columns),
                column2=list(df.select_dtypes('number').columns)):
    print(f"Correlation: {df[column1].corr(df[column2])}")

In [ ]:
import cufflinks as cf
cf.go_offline(connected=True)

@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), 
             text='title',
             title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)

In [ ]:
def plot_up_to(column, date):
    date = pd.Timestamp(date)
    plot_df = df.loc[df.index <= date].copy()
    plot_df[column].cumsum().iplot(mode='markers+lines', 
                                   xTitle='published date',
                                   yTitle=column, 
                                  title=f'Cumulative {column.title()} Until {date.date()}')
    
_ = interact(plot_up_to, column=widgets.Dropdown(options=list(df.select_dtypes('number').columns)), 
             date = widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

In [ ]:
directory = widgets.Dropdown(options=['images', 'screenshots'])
images = widgets.Dropdown(options=os.listdir(directory.value))

def update_images(*args):
    images.options = os.listdir(directory.value)
    
directory.observe(update_images, 'value')

def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))
    
_ = interact(show_images, fdir=directory, file=images)
